# Chapter XVI: Binary Logistic Regression

In [1]:
import matplotlib.pyplot as plt
import nnfs
from nnfs.datasets import spiral_data
import numpy as np
nnfs.init()

In [9]:
import sys, os

# Go up one directory and into 'notebooks'
sys.path.append(os.path.abspath('../modules'))

from layers import Layer_Dense
from activation_functions import Activation_ReLU, Activation_Sigmoid
from losses import Activation_Softmax_Loss_CategoricalCrossentropy, Loss_BinaryCrossentropy
from optimizers import Optimizer_Adam

In [2]:
# Create dataset
X, y = spiral_data(samples=100, classes=2)

In [3]:
# Reshape labels to be a list of lists
# Inner list contains one output (either 0 or 1)
# per each output neuron, 1 in this case
y = y.reshape(-1, 1)
print(y[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [10]:
# Create dataset
X, y = spiral_data(samples=100, classes=2)
# Reshape labels to be a list of lists
# Inner list contains one output (either 0 or 1)
# per each output neuron, 1 in this case
y = y.reshape(-1, 1)
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64, weight_regularizer_l2=5e-4,
bias_regularizer_l2=5e-4)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 1 output value
dense2 = Layer_Dense(64, 1)
# Create Sigmoid activation:
activation2 = Activation_Sigmoid()
# Create loss function
loss_function = Loss_BinaryCrossentropy()
# Create optimizer
optimizer = Optimizer_Adam(decay=5e-7)
# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function
    # of first layer as inputs
    dense2.forward(activation1.output)
    # Perform a forward pass through activation function
    # takes the output of second dense layer here
    activation2.forward(dense2.output)
    # Calculate the data loss
    data_loss = loss_function.calculate(activation2.output, y)
    # Calculate regularization penalty
    regularization_loss = \
    loss_function.regularization_loss(dense1) + \
    loss_function.regularization_loss(dense2)
    # Calculate overall loss
    loss = data_loss + regularization_loss
    # Calculate accuracy from output of activation2 and targets
    # Part in the brackets returns a binary mask - array consisting
    # of True/False values, multiplying it by 1 changes it into array
    # of 1s and 0s
    predictions = (activation2.output > 0.5) * 1
    accuracy = np.mean(predictions==y)
    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
        f'acc: {accuracy:.3f}, '+
        f'loss: {loss:.3f} (' +
        f'data_loss: {data_loss:.3f}, ' +
        f'reg_loss: {regularization_loss:.3f}), ' +
        f'lr: {optimizer.current_learning_rate}')
    # Backward pass
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

epoch: 0, acc: 0.540, loss: 0.693 (data_loss: 0.693, reg_loss: 0.000), lr: 0.001
epoch: 100, acc: 0.605, loss: 0.674 (data_loss: 0.673, reg_loss: 0.001), lr: 0.0009999505024501287
epoch: 200, acc: 0.605, loss: 0.670 (data_loss: 0.669, reg_loss: 0.001), lr: 0.0009999005098992651
epoch: 300, acc: 0.595, loss: 0.666 (data_loss: 0.665, reg_loss: 0.002), lr: 0.000999850522346909
epoch: 400, acc: 0.595, loss: 0.662 (data_loss: 0.660, reg_loss: 0.002), lr: 0.0009998005397923115
epoch: 500, acc: 0.600, loss: 0.658 (data_loss: 0.655, reg_loss: 0.003), lr: 0.0009997505622347225
epoch: 600, acc: 0.605, loss: 0.652 (data_loss: 0.648, reg_loss: 0.004), lr: 0.0009997005896733929
epoch: 700, acc: 0.595, loss: 0.644 (data_loss: 0.639, reg_loss: 0.005), lr: 0.0009996506221075735
epoch: 800, acc: 0.645, loss: 0.632 (data_loss: 0.624, reg_loss: 0.008), lr: 0.000999600659536515
epoch: 900, acc: 0.705, loss: 0.616 (data_loss: 0.605, reg_loss: 0.011), lr: 0.0009995507019594694
epoch: 1000, acc: 0.730, loss:

In [11]:
# Validate the model
# Create test dataset
X_test, y_test = spiral_data(samples=100, classes=2)
# Reshape labels to be a list of lists
# Inner list contains one output (either 0 or 1)
# per each output neuron, 1 in this case
y_test = y_test.reshape(-1, 1)
# Perform a forward pass of our testing data through this layer
dense1.forward(X_test)
# Perform a forward pass through activation function
# takes the output of first dense layer here
activation1.forward(dense1.output)
# Perform a forward pass through second Dense layer
# takes outputs of activation function of first layer as inputs
dense2.forward(activation1.output)
# Perform a forward pass through activation function
# takes the output of second dense layer here
activation2.forward(dense2.output)
# Calculate the data loss
loss = loss_function.calculate(activation2.output, y_test)
# Calculate accuracy from output of activation2 and targets
# Part in the brackets returns a binary mask - array consisting of
# True/False values, multiplying it by 1 changes it into array
# of 1s and 0s
predictions = (activation2.output > 0.5) * 1
accuracy = np.mean(predictions==y_test)
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

validation, acc: 0.940, loss: 0.209
